In [ ]:
#Bibliothek load
import json
import nltk
from nltk.corpus import wordnet
import plotly.figure_factory as ff
import numpy as np
import matplotlib.pyplot as plt
from graphviz import Graph
import pandas as pd
from anytree import Node, RenderTree
from anytree.exporter import DotExporter
import sys


#Basis Hypernym Funktion für die Extraktion des Hypernymes
def get_hypernym(syns):
    hypernymList = []
    paths = wordnet.synset(syns).hypernym_paths() #all Hypernym extract
    for weg in paths:
        hypernymList = hypernymList + [synset.name() for synset in weg]
    #print(paths)
    hypernymList = list(dict.fromkeys(hypernymList )) #remove duplicate
    return hypernymList
def get_directhypernym(syns):
    hypernymList = [hyp.name() for hyp in  wordnet.synset(syns).hypernyms()]
    return hypernymList

In [ ]:
#Hier werden alle benötigten Wörtebucher erstellt

#Mapping des Ideen mit nummerId und Content
def get_dictionaryIdee(ideen):
    ideenDict = {}  # key = (nummerId,Id) Value = Content
    nummer = 1
    for idee in ideen:  #jeder idee durchlaufen
        ideenDict[(nummer,idee["id"])] = idee.get("content")
        nummer = nummer+1
    return ideenDict

#----------------------------------------------------------------------------------------------------------------
#SynsetsDict mit der Ausgewählten Synsets bei der Annotation
def get_synsetsList(ideen):
    synsetsDict  = {} # key = Synset Value = [list von ID]
    for idee in ideen:  #jeder idee durchlaufen
        annotiert = idee.get("annotations")
        if annotiert :
            for candidate in idee['annotations']: #word in eine idee  
                if candidate['validated'] == True: #wenn diese word annotiert ist
                    for resources in candidate['resource_candidates']: #annotation candidates durchlaufen
                        if resources["selected"] == True: #wenn ein kandidates ausgewählt ist
                            if not synsetsDict.get(resources["source"]): #wenn diese synsets noch nicht existiert
                                synsetsDict[resources["source"]] = []  # fügt diese synsets in synsetList
                                synsetsDict[resources["source"]].append(idee['id']) #fügt die idee_id die diese synset beinhaltet
                            else:
                                synsetsDict[resources["source"]].append(idee['id']) #idee_id einfügen           
                else:
                    print("not choice")
        else:
            print("candidate was not annotiert")
    return synsetsDict

#----------------------------------------------------------------------------------------------------------------
#Hypernym Wörterbuch mit der Hypernyme der behinhaltet Ideen ID 
def get_hypernymDict(synsetsDict):
    hypernymDict = {} #Key = hypernym , Value = [ListId]
    #for idee in ideen_list:  #jeder idee durchlaufen 
    for key in synsetsDict.keys(): #hypernym von jeder synsets builden
            hypernymList = get_hypernym(key)
            if hypernymList:
                for hypernym in hypernymList:
                    if not hypernymDict.get(hypernym): #wenn das Hypernym noch nicht existiert
                            hypernymDict[hypernym]= []
                            hypernymDict[hypernym].extend(synsetsDict[key]) #alle idee_id in der zugehörigen hypernym einfügen
                 
                    else: #sonst fügt die idee id nur
                            hypernymDict[hypernym].extend(synsetsDict[key])
                    hypernymDict[hypernym] = list(dict.fromkeys(hypernymDict[hypernym] )) #remove duplicate
                    
    return hypernymDict
                



#------------------------------------------------------------------------------------------------------------------------------------
#Hilfsfunktion für DictionaryTree Extrahiert nach einander des DirectHypernymen des vorhandenen Hypernymen(Synsets) 
def build_dictionary(dictionary_actual):
    Anzahl = 0
    for element in dictionary_actual.copy():
        if element == "entity.n.01":
            pass
        else:
            hypernymList = get_directhypernym(element)
            if hypernymList == []: #wenn kein direct hypernym mehr vorhanden ist. bzw. ist schon entity
                Anzahl= Anzahl + 1
                #print("elem ist: ", element)
                if "entity.n.01" in dictionary_actual:
                    dictionary_actual["entity.n.01"].append(element)
                else:
                    dictionary_actual["entity.n.01"] = []
                    dictionary_actual["entity.n.01"].append(element)
            else:
                for hypernym in hypernymList:
                    if hypernym in dictionary_actual:
                        dictionary_actual[hypernym].append(element)
                    else:
                        dictionary_actual[hypernym] = []
                        dictionary_actual[hypernym].append(element)
    print ("entity", Anzahl)    
    return dictionary_actual

#dictionaryTree: Beziehungen zwischen alle Hypernymen
def analyse(synsetsDict):
    #f = open(filename,'w')
    dictionaryTree = {} #schon bearbeiten knoten #key = hypernyme Value = [List von Hypernymen die als directHypernym der Key haben]
    dictionary_actual = {} #contains new Hypernym which is not in dictionaryTree
    #distance_matrix = 
    for syns in synsetsDict.keys(): #hypernym von jeder synsets builden
        dictionary_actual[syns] = []
        dictionaryTree[syns]    = []
                
    while(len(dictionary_actual) != 0):
        dictionary_actual = build_dictionary(dictionary_actual)
        for elem in dictionary_actual.copy():
            if elem in dictionaryTree: #wenn diese elem schon vorhanden ist
                dictionaryTree[elem].extend(dictionary_actual.pop(elem)) #fügt einfach die leaves hinzu und lösch diese aus dem actual schon bearbeiten
                dictionaryTree[elem] = list(dict.fromkeys(dictionaryTree[elem])) #remove duplicate
            else:
                dictionaryTree[elem] = [] #wenn nicht erstellt einen neue Eltern knoten
                dictionaryTree[elem].extend(dictionary_actual[elem])
                #dictionaryTree[elem] = dictionary_actual[elem] #fügt die leaves davon            
    return dictionaryTree




#----------------------------------------------------------------------------------------------------------------
#ideen_synsetsDict: Pro Ideen alle behinhaltetet Synsets
def mapping_idee_to_synsets(ideen):
    ideen_synsetsDict = {}  # key = (nummerId,Id) Value = [Listsynsets]
    nummer = 1
    for idee in ideen:  #jeder idee durchlaufen
        ideen_synsetsDict[(nummer,idee["id"])] = []
        annotiert = idee.get("annotations")
        if annotiert :
            for candidate in idee['annotations']: #word in eine idee  
                if candidate['validated'] == True: #wenn diese word annotiert ist
                    for resources in candidate['resource_candidates']: #annotation candidates durchlaufen
                        if resources["selected"] == True: #wenn ein kandidates ausgewählt ist
                            ideen_synsetsDict[(nummer,idee["id"])].append(resources["source"])
        nummer = nummer +1 
    return ideen_synsetsDict



#----------------------------------------------------------------------------------------------------------------
#ideen_hypernymDict: Pro Idee alle behinhatetet Hypernymen
def mapping_idee_to_hypernym(ideen_synsetsDict):
    ideen_hypernymDict = {} # key = (nummerId,Id)  Value = [Listhypernymen]
    for elem in ideen_synsetsDict:
        ideen_hypernymDict[elem] = []
        for synset in ideen_synsetsDict[elem]:
            hypernymList = get_hypernym(synset)
            ideen_hypernymDict[elem].extend(hypernymList)
        ideen_hypernymDict[elem] = list(dict.fromkeys(ideen_hypernymDict[elem] )) #remove duplicate
    return ideen_hypernymDict



#----------------------------------------------------------------------------------------------------------------
#occurence_synsetsDict: Anzahl an Occurence von einen Synset in der Datensatzt     
def occurence_of_synset(ideen_synsetDict):
    occurence_synsetsDict = {} #key = synset Value = occurence
    for synslist in ideen_synsetDict.values():
        for syns in synslist:
            if not (syns in occurence_synsetsDict.keys()):
                occurence_synsetsDict[syns] = 0
            occurence_synsetsDict[syns] = occurence_synsetsDict[syns] + 1
    return occurence_synsetsDict
        
        
    
        
    

In [ ]:

#Jeder Ideen auf einen Konzept reduzieren Konzept die am wenigstens in der Datensatz vorkommen genau 1% 
#Beim mehrere Konzept pro Ideen mit der wiederholung Prozent gleich 1% wählen wir einfach der Erste
def Prozent(tupel,i):
    prozent = ((((tupel[-1][1])*i)//100)+1)
    return prozent
def ideen_synsetDict_clearly(ideen_synsetsDict, occurence_synsetsDict): #Mapping Ideen zu besten Synset
    ideen_synsetdict_clearly = {}
    tupel  = [(k, v) for k, v in occurence_synsetsDict.items()]
    tupel.sort(key = sort_by_keys)
    kleiner = [k for k,i in tupel if i<=Prozent(tupel,1)]
    for i,j in ideen_synsetsDict.items():
        synsets = [s for s in j if s in kleiner]
        synsets = list(dict.fromkeys(synsets)) #duplicate löschen
        ideen_synsetdict_clearly[i] = []
        if len(synsets) >= 1 :   #ideen die Mehr als 1 besten synset hat einfach die erste auswählen.
            ideen_synsetdict_clearly[i].append(synsets[0])
        else: #ideen die kein Inhalt mehr haben(Grund alle Synsets existiert schon in einer andere Idee) 
            a = 2
            while len(synsets) == 0: #Anzahl der wiederholung erhöhen und weiter suchen
                kleiner2 = [k for k,i in tupel if i<=Prozent(tupel,a)]
                synsets = [s for s in j if s in kleiner2]
                synsets = list(dict.fromkeys(synsets)) #duplicate löschen
                a=a+1
            ideen_synsetdict_clearly[i].append(synsets[0])          
    
    return ideen_synsetdict_clearly #key = (nummerId,Id), Value = [ein Synset]

def synsetDict_clearly(ideen_synsetdict_clearly, synsetsDict ): #new SynsetDict mit besten Synset
    for synset in synsetsDict.copy():
        find = False 
        for item in ideen_synsetdict_clearly.values():
            if synset in item:
                find = True
        if find == False:
            synsetsDict.pop(synset)
    return synsetsDict
    

In [ ]:
#darstellung des DictionaryTree in einen Baum Form
def build_tree(dictionaryTree):
    #anzahl = 0
    parent = Graph('parent')
    for elem in dictionaryTree:
        for node in dictionaryTree[elem]:
            parent.edge(elem,node,len='1.00')
    leaves = Graph('child')
    leaves.attr(rank='same')
    for elem in dictionaryTree.keys():
        if dictionaryTree[elem] == []:
            #anzahl = anzahl + 1
            leaves.node(elem)
    #print(anzahl)
    parent.subgraph(leaves)
    parent.view()

    
def build_part_of_tree(Tree,knoten):
    s = Graph(knoten)
    for node in Tree[knoten]:
        s.edge(knoten,node)
    s.view()
    
    
    
    

In [ ]:
#Balkendiagramm Darstellung des vorhandenen Dictionary

def sort_by_keys(val):
    return val[1]

#https://matplotlib.org/examples/api/barchart_demo.html
def autolabel(rects,ax):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom')



#https://bodo-schoenfeld.de/jupyter-notebook-balkendiagramm-erstellen/
def plot_balken_hypernym(hypernymdict,prozent,prozentr):
    fig, ax = plt.subplots()
    tupel2 = [(k, len(v)) for k, v in hypernymdict.items()]
    tupel2.sort(key = sort_by_keys)
    prozent2 = ((tupel2[-1][1] * prozent)/100)
    prozent3 = ((tupel2[-1][1] * prozentr)/100)
    kleiner = [k for k,i in tupel2 if i<=prozent2]
    groesser = [k for k,i in tupel2 if i>=prozent3]
    #print("Others: ", kleiner)
    tupel = [(k,v) for k,v in tupel2 if k not in kleiner]
    tupel = [(k,v) for k,v in tupel if k not in groesser]
    
    length2 = len(tupel) 
    #print(length2)
    tupel.insert(0,("Others",prozent2))
    tupel.insert(length2+1,("Others2",prozent3))
    
    hypernymlist = [i[0] for i in tupel]
    anteile = [i[1] for i in tupel]
    anteile.sort()
    index = np.arange(len(hypernymlist))
    bar_width = 0.5
    bars = plt.bar(index, anteile,bar_width,color="green")
    plt.xticks(index, hypernymlist,fontsize=20)
    plt.title("Anzahl von Ideen pro Hypernym",fontsize=30)
    plt.ylabel('Anzahl von Ideen',fontsize=30)
    plt.xlabel("Hypernym",fontsize=30)
    length = anteile[-1] + 5
    plt.ylim(0,length) #y-achse höhe
    ax.set_xticks(range(len(hypernymlist)))
    ax.set_xticklabels(hypernymlist, rotation='vertical')
    fig_size = plt.gcf().get_size_inches() #Get current size
    sizefactor = 3 #Set a zoom factor
    autolabel(bars,ax)
    plt.gcf().set_size_inches(sizefactor * fig_size) # Modify the current size by the factor
    #plt.show()
    plt.savefig('balken_hypernym.pdf')
    
def plot_hypernyme_fenster(hypernymdict,minimum,maximum): #gute Liste von Hypernyme Anzahl an ideen zwischen min-max
    fig, ax = plt.subplots()
    tupel2 = [(k, len(v)) for k, v in hypernymdict.items()]
    tupel2.sort(key = sort_by_keys)
    tupel = [(k,v) for k,v in tupel2 if minimum <= v <= maximum] #k= hyp und v = anzahl
    Anzahlhypernyme = []
    Abdeckung = []
    Anzahlidee = []
    minimum_h = minimum
    while minimum_h <= maximum:
        tupel3 = [(k,v) for k,v in tupel if minimum_h == v]
        if len(tupel3)<=1:
               # A = []
                pass
        else:
            A = set(hypernymdict[tupel3[0][0]]).intersection(set(hypernymdict[tupel3[1][0]]))
            if len(tupel3)>2:
                for i in range (2,len(tupel3)):
                    A = A.intersection(set(hypernymdict[tupel3[i][0]]))
            if (len(A)>0):
                Abdeckung.append(len(A))
                Anzahlhypernyme.append(len(tupel3))
                Anzahlidee.append(minimum_h)
        minimum_h = minimum_h + 1
   # print(Abdeckung)
   # print(Anzahlhypernyme)
   # print([i for i in range (minimum, maximum+1)])
        
    tabelle = pd.DataFrame({
        'Anzahl an Ideen':pd.Categorical([i for i in Anzahlidee]),                                 
        'Überschneidung' :pd.Categorical(Abdeckung),
        'Anzahlhypernyme':pd.Categorical([i for i in Anzahlhypernyme])
    })
    tabelle.to_excel("HypernymFenster.xlsx")
    #print(tabelle)
    #newtupel = 
    hypernymlist = [i[0] for i in tupel if i[1] in Anzahlidee]
    anteile = [i[1] for i  in tupel if i[1] in Anzahlidee]
    anteile.sort()
    #print(anteile)
    index = np.arange(len(hypernymlist))
    bar_width = 0.5
    bars = plt.bar(index, anteile,bar_width,color="green")
    plt.xticks(index, hypernymlist,fontsize=20)
    plt.title("Hypernyme mit Anzahl an Ideen zwischen " + str(minimum) + " und " + str(maximum) + " inklusive",fontsize=25)
    plt.ylabel('Anzahl von Ideen',fontsize=30)
    plt.xlabel("Hypernym",fontsize=30)
    length = anteile[-1] + 5
    plt.ylim(0,length) #y-achse höhe
    ax.set_xticks(range(len(hypernymlist)))
    ax.set_xticklabels(hypernymlist, rotation='vertical')
    fig_size = plt.gcf().get_size_inches() #Get current size
    sizefactor = 3 #Set a zoom factor
    autolabel(bars,ax)
    plt.gcf().set_size_inches(sizefactor * fig_size) # Modify the current size by the factor
    plt.show()
    
def plot_mapping_idee_Synstet(ideen_synsetDict,prozent):
    fig, ax = plt.subplots()
    tupel2 = [(k[0], len(v)) for k, v in ideen_synsetDict.items()]
    tupel2.sort(key = sort_by_keys)
    prozent2 = ((tupel2[-1][1] * prozent)/100)
    kleiner = [k for k,i in tupel2 if i<=prozent2]
    print("Others: ", kleiner)
    tupel = [(k,v) for k,v in tupel2 if k not in kleiner]
    tupel.insert(0,("Others",prozent2))
    ideen_id = [i[0] for i in tupel]
    anteile  = [i[1] for i in tupel]
    anteile.sort()
    index = np.arange(len(ideen_id))
    bars2 = plt.bar(index, anteile,color="green")
    plt.xticks(index, ideen_id,fontsize=20)
    plt.title("Anzahl von Synset pro Idee",fontsize=30)
    plt.ylabel('Anzahl von Synsets',fontsize=30)
    plt.xlabel("Idee Id",fontsize=30)
    length = anteile[-1] + 5
    plt.ylim(0,length) #y-achse höhe
    ax.set_xticks(range(len(ideen_id)))
    ax.set_xticklabels(ideen_id, rotation='vertical')
    fig_size = plt.gcf().get_size_inches() #Get current size
    sizefactor = 3 #Set a zoom factor
    autolabel(bars2, ax)
    plt.gcf().set_size_inches(sizefactor * fig_size) # Modify the current size by the factor
    plt.show()
    
def plot_mapping_idee_Hypernym(ideen_hypernymDict,prozent):
    fig, ax = plt.subplots()
    tupel2 = [(k[0], len(v)) for k, v in ideen_hypernymDict.items()]
    tupel2.sort(key = sort_by_keys)
    prozent2 = ((tupel2[-1][1] * prozent)/100)
    kleiner = [k for k,i in tupel2 if i<=prozent2]
    #print(kleiner)
    print("Others: ", kleiner)
    tupel = [(k,v) for k,v in tupel2 if k not in kleiner]
    tupel.insert(0,("Others",prozent2))
    ideen_id = [i[0] for i in tupel]
    anteile  = [i[1] for i in tupel]
    anteile.sort()
    index = np.arange(len(ideen_id))
    bars3 = plt.bar(index, anteile,color="green")
    plt.xticks(index,ideen_id,fontsize=20)
    plt.title("Anzahl von Hypernym pro Idee",fontsize=30)
    plt.ylabel('Anzahl von Hypernyme',fontsize=30)
    plt.xlabel("Idee Id",fontsize=30)
    length = anteile[-1] + 5
    plt.ylim(0,length) #y-achse höhe
    ax.set_xticks(range(len(ideen_id)))
    ax.set_xticklabels(ideen_id, rotation='vertical')
    fig_size = plt.gcf().get_size_inches() #Get current size
    sizefactor = 3 #Set a zoom factor
    plt.gcf().set_size_inches(sizefactor * fig_size) # Modify the current size by the factor
    autolabel(bars3,ax)
    plt.show()

def plot_occurence_synset(occurence_synsetsDict,prozent):
    fig, ax = plt.subplots()
    tupel2  = [(k, v) for k, v in occurence_synsetsDict.items()]
    tupel2.sort(key = sort_by_keys)
    prozent2 = ((tupel2[-1][1] * prozent)/100)
    kleiner = [k for k,i in tupel2 if i<=prozent2]
    print("Others: ", kleiner)
    tupel   = [(k,v) for k,v in tupel2 if k not in kleiner]
    tupel.insert(0,("Others",prozent2))
    synsets = [i[0] for i in tupel]
    anteile = [i[1] for i in tupel]
    anteile.sort()
    index = np.arange(len(synsets))
    bar_width = 0.5
    bars = plt.bar(index, anteile,bar_width,color="green")
    plt.xticks(index, synsets,fontsize=20)
    plt.title("Anzahl von Occurence eines Synset",fontsize=30)
    plt.ylabel('Occurence',fontsize=30)
    plt.xlabel("Synsets",fontsize=30)
    length = anteile[-1] + 5
    plt.ylim(0,length) #y-achse höhe
    ax.set_xticks(range(len(synsets)))
    ax.set_xticklabels(synsets, style='oblique',rotation='vertical')
    fig_size = plt.gcf().get_size_inches() #Get current size
    sizefactor = 3 #Set a zoom factor
    autolabel(bars,ax)
    plt.gcf().set_size_inches(sizefactor * fig_size) # Modify the current size by the factor
    plt.show()
    


In [ ]:

#Abstractionsgrad: 
def Abstractionsgradfunc(ideen_synsetsDict): 
    AbstractionsDict = {} #key = (numId,Id) value = distance
    for key in ideen_synsetsDict.keys():
        AbstractionsDict[key] = 0
        synsetsdist = []
        for syns in ideen_synsetsDict[key]:
            paths = wordnet.synset(syns).hypernym_paths() #all Hypernym extract
            distList = [len(i) for i in paths]
            dist = sum(distList)/len(paths)         #Distance von einen Synset bis zur root
            synsetsdist.append(dist)
        AbstractionsDict[key] = sum(synsetsdist)/len(ideen_synsetsDict[key])
    return AbstractionsDict

def Top_Abstract_and_concret(AbstractionsDict,ideenDict,top):
    tupel2 = [(k[0],v) for k, v in AbstractionsDict.items()]
    tupel2.sort(key = sort_by_keys)
    top_Konkreten    = [nummer for (nummer,v) in AbstractionsDict.items() if (nummer[0],v) in tupel2[-top:]]
    top_Abstrackten  = [nummer for (nummer,v) in AbstractionsDict.items() if (nummer[0],v) in tupel2[0:top]]
    listTop_KonKret  = [(nummer[0],c) for (nummer,c) in ideenDict.items() if nummer in top_Konkreten]
    listTop_Abstrakt = [(nummer[0],c) for (nummer,c) in ideenDict.items() if nummer in top_Abstrackten]
    Ideen_ID         = [nummer[0] for nummer in tupel2[0:top]] + [nummer[0] for nummer in tupel2[-top:]]
    
    tabelle = pd.DataFrame({
        'ID von Ideen':pd.Categorical(Ideen_ID),                                 
        'Abstraktionsgrad' :pd.Categorical([v for (k,v) in tupel2[0:top]] + [v for (k,v) in tupel2[-top:]]),
        'Inhalt' :pd.Categorical([i[1] for i in listTop_Abstrakt] + [i[1] for i in listTop_KonKret]) 
    })
    tabelle.to_excel("Abstraktionsgrad.xlsx")
    #print(tabelle)
    return listTop_Abstrakt,listTop_KonKret

   
    


In [ ]:

# read file
if __name__ == '__main__':
    
    #data loaded
    with open('export200.json', 'r') as myfile:
        data = myfile.read()
    ideen_list   = json.loads(data)
    
    #Dictionary build
    synsetsList  = get_synsetsList(ideen_list)
    hypernymList = get_hypernymDict(synsetsList)
    ideenDict    = get_dictionaryIdee(ideen_list)
    print("Anzahl Synsets", len(synsetsList.keys()))
    print("Anzahl Hypernym", len(hypernymList.keys()))
    ideen_synset   = mapping_idee_to_synsets(ideen_list)
    ideen_hypernym = mapping_idee_to_hypernym(ideen_synset)
    synsets_occurence = occurence_of_synset(ideen_synset)
    ideen_synsetList_clearly = ideen_synsetDict_clearly(ideen_synset, synsets_occurence)
    synsetList_clearly = synsetDict_clearly(ideen_synsetList_clearly, synsetsList )
    hypernymList_clearly = get_hypernymDict(synsetList_clearly)
    Abstractionsgrad = Abstractionsgradfunc(ideen_synset)
    listTop5_Abstrakt,listTop5_KonKret = Top_Abstract_and_concret(Abstractionsgrad,ideenDict,10)
    print(listTop5_Abstrakt)
    print(listTop5_KonKret)
    print("Anzahl Synsets clearly", len(synsetList_clearly.keys()))
    print("Anzahl Hypernym clearly", len(hypernymList_clearly.keys()))
    
    
    #Ploting
    plot_balken_hypernym(hypernymList_clearly,2.5,30)
    plot_hypernyme_fenster(hypernymList,1,200)
    plot_mapping_idee_Synstet(ideen_synset,30)
    plot_mapping_idee_Hypernym(ideen_hypernym, 30)
    plot_occurence_synset(synsets_occurence,10)
    
    
    
    #Tree
    dictionaryTree = analyse(synsetsList)
    dictionaryTree_clearly = analyse(synsetList_clearly)
    #build_tree(dictionaryTree)
    #build_tree(dictionaryTree_clearly)
    #build_tree_entity(dictionaryTree)
    #build_tree_physical_entity(dictionaryTree)
    #build_tree_entity(dictionaryTree_clearly)
    #build_tree2(dictionaryTree_clearly)
    #add_leaves(dictionaryTree)
    ''''print("entity list", (dictionaryTree['entity.n.01']))
    print(len(dictionaryTree_clearly['entity.n.01']))
    print((dictionaryTree['physical_entity.n.01']))
    print((dictionaryTree_clearly['physical_entity.n.01']))
    print("erste", (dictionaryTree['measure.n.02']))
    print("erste", (dictionaryTree_clearly['measure.n.02']))
    print("zweite",(dictionaryTree['communication.n.02']))
    print("zweite",(dictionaryTree_clearly['communication.n.02']))
    print("drei", (dictionaryTree['attribute.n.02']))
    print("drei", (dictionaryTree_clearly['attribute.n.02']))
    print("Vier", (dictionaryTree['relation.n.01']))
    print("Vier", (dictionaryTree_clearly['relation.n.01']))
    print("fünf", (dictionaryTree['group.n.01']))
    print("fünf", (dictionaryTree_clearly['group.n.01']))
    print("abstr_clear", (dictionaryTree_clearly['psychological_feature.n.01']))
    print("abstr", (dictionaryTree['psychological_feature.n.01']))
    #print("abstr", (ideen_hypernym['causal_agent.n.01']))
    casual_agent = [ideenDict[(i,id1)] for (i,id1) in ideenDict.keys() if id1 in hypernymList['causal_agent.n.01']]  
    print(len(casual_agent))'''
    
    
    
    #build_part_of_tree(dictionaryTree, "physical_entity.n.01")
   
  
    